### 1. Carregando as Bibliotecas necessárias.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

### 2. Carregando os dados brutos.

In [4]:
print("Carregando o arquivo 'riser_pq_uni.csv'...")
raw_df = pd.read_csv('riser_pq_uni.csv')
print("Dados carregados com sucesso!")
print("\nPrimeiras 5 linhas dos dados brutos:")
raw_df.head()

Carregando o arquivo 'riser_pq_uni.csv'...
Dados carregados com sucesso!

Primeiras 5 linhas dos dados brutos:


,Time (sec),Pressure (bar) @x_{99}=56.9453,Pressure (bar) @x_{105}=60.4141,Pressure (bar) @x_{109}=62.7266,Pressure (bar) @x_{114}=65.6172,Pressure (bar) @x_{119}=68.5078,Pressure (bar) @x_{124}=71.3984,Pressure (bar) @x_{128}=73.7109,Gas mass rate (kg/sec),Liquid mass rate (kg/sec)
0,0.00000,2.0689,2.0210,1.8318,1.6007,1.3771,1.1628,1.0000,0.015360,40.085
1,0.29493,2.0645,2.0420,1.8483,1.6142,1.3887,1.1739,1.0125,0.031328,81.781
2,0.59320,2.0643,2.0459,1.8565,1.6258,1.4022,1.1881,1.0198,0.024000,61.016
3,0.89486,2.0671,2.0470,1.8617,1.6317,1.4083,1.1931,1.0272,0.016026,39.192
4,1.19880,2.0694,2.0490,1.8635,1.6326,1.4085,1.1929,1.0322,0.009156,21.469


### 3. Pre-Processamento dos dados.

In [5]:

# --- 2. Renomear as Colunas ---
# Mapeia os nomes antigos para os novos para facilitar o acesso
column_mapping = {
    'Time (sec)': 'time',
    'Pressure (bar) @x_{99}=56.9453': 'pressure_1',
    'Pressure (bar) @x_{105}=60.4141': 'pressure_2',
    'Pressure (bar) @x_{109}=62.7266': 'pressure_3',
    'Pressure (bar) @x_{114}=65.6172': 'pressure_4',
    'Pressure (bar) @x_{119}=68.5078': 'pressure_5',
    'Pressure (bar) @x_{124}=71.3984': 'pressure_6',
    'Pressure (bar) @x_{128}=73.7109': 'pressure_7',
    'Gas mass rate (kg/sec)': 'gas_flow_rate',
    'Liquid mass rate (kg/sec)': 'liquid_flow_rate'
}
df = raw_df.rename(columns=column_mapping)
print("\nColunas renomeadas com sucesso!")

# --- 3. Separar Features (X) e Alvo (y) ---
# Vamos focar em prever a vazão de líquido por enquanto
features = ['pressure_1', 'pressure_2', 'pressure_3', 'pressure_4', 'pressure_5', 'pressure_6', 'pressure_7']
target = 'liquid_flow_rate'

X = df[features]
y = df[target]

# --- 4. Divisão em Dados de Treino e Teste ---
# Vamos usar 80% dos dados para treino e 20% para teste
# 'shuffle=False' é importante para séries temporais, para não misturar a ordem dos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)
print(f"\nDados divididos: {len(X_train)} amostras de treino e {len(X_test)} de teste.")

# --- 5. Escalonamento dos Dados (Normalization) ---
# Criar um scaler para as features
scaler_X = MinMaxScaler()
# Criar um scaler para o alvo
scaler_y = MinMaxScaler()

# Ajustar o scaler APENAS nos dados de treino para evitar vazamento de dados do teste
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

# Aplicar a mesma transformação aos dados de teste
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

print("\nEscalonamento Min-Max aplicado com sucesso.")

# --- 6. Juntar os Dados e Salvar em CSV ---
# Criar DataFrames com os dados processados
train_scaled_df = pd.DataFrame(X_train_scaled, columns=features)
train_scaled_df['time'] = df.loc[X_train.index, 'time'].values
train_scaled_df['liquid_flow_rate'] = y_train_scaled.flatten() # Achatamos o array para uma única coluna

test_scaled_df = pd.DataFrame(X_test_scaled, columns=features)
test_scaled_df['time'] = df.loc[X_test.index, 'time'].values
test_scaled_df['liquid_flow_rate'] = y_test_scaled.flatten()

# Salvar em novos arquivos CSV
train_scaled_df.to_csv('train_data_scaled_manual.csv', index=False)
test_scaled_df.to_csv('test_data_scaled_manual.csv', index=False)

print("\nArquivos 'train_data_scaled_manual.csv' e 'test_data_scaled_manual.csv' criados com sucesso!")


Colunas renomeadas com sucesso!

Dados divididos: 28295 amostras de treino e 7074 de teste.

Escalonamento Min-Max aplicado com sucesso.

Arquivos 'train_data_scaled_manual.csv' e 'test_data_scaled_manual.csv' criados com sucesso!
